# Fraud Detecteion (With SQL)

In [5]:
import numpy as np 
import pandas as pd
from sqlalchemy import create_engine

# File path and database configuration
csv_file_path = '/Users/teitelbaumsair/Desktop/Data Bootcamp Repo/DI_Bootcamp/Data Bootcamp Final Project/Data/DataCoSupplyChainDataset_clean.csv'
db_config = {
    "username": "postgres",
    "password": "1234",
    "host": "localhost",  
    "port": 5433,        
    "database": "brazzil_ecommerce"
}

# Create engine
engine = create_engine(
    f"postgresql+psycopg2://{db_config['username']}:{db_config['password']}@"
    f"{db_config['host']}:{db_config['port']}/{db_config['database']}"
)

# Table name
table_name = 'supply_chain_data'

try:
    # Read CSV file into DataFrame
    df = pd.read_csv(csv_file_path)
    # Load DataFrame into PostgreSQL table
    df.to_sql(table_name, con=engine, index=False, if_exists='replace')  
    print(f"Successfully loaded {table_name}")
except Exception as e:
    print(f"Error processing {csv_file_path} for table {table_name}: {e}")

print(f"Data successfully inserted into the table '{table_name}' in PostgreSQL.")

Successfully loaded supply_chain_data
Data successfully inserted into the table 'supply_chain_data' in PostgreSQL.


Fraud Detection

In [28]:
year_month = '2018-01'
shipping_mode = 'First Class'
suspected_fraud = 'True'

# Adjusted query to include both first and last customer names
query = f"""
SELECT "Customer Fname", "Customer Lname",
       COUNT("Order Id") AS order_count, 
       (COUNT("Order Id") * 100.0 / 
        (SELECT COUNT(*) 
         FROM supply_chain_data
         WHERE TO_CHAR("order date (DateOrders)"::timestamp, 'YYYY-MM') = '{year_month}')
       ) AS percentage
FROM supply_chain_data
WHERE TO_CHAR("order date (DateOrders)"::timestamp, 'YYYY-MM') = '{year_month}'
      AND "Shipping Mode" = '{shipping_mode}'
      AND "SUSPECTED_FRAUD" = {suspected_fraud}
GROUP BY "Customer Fname", "Customer Lname"
ORDER BY order_count DESC;
"""

# Execute the query and store the result in a DataFrame
try:
    result = pd.read_sql_query(query, con=engine)
    display(result)
except Exception as e:
    print(f"Error executing query: {e}")

,Customer Fname,Customer Lname,order_count,percentage
0,Erica,Sutton,1,0.047103
1,Ingrid,Bradley,1,0.047103
2,Reagan,Gallegos,1,0.047103
3,Yen,Jacobson,1,0.047103


In [29]:
year_month = '2018-01'
shipping_mode = 'Second Class'
suspected_fraud = 'True'

# Adjusted query to include both first and last customer names
query = f"""
SELECT "Customer Fname", "Customer Lname",
       COUNT("Order Id") AS order_count, 
       (COUNT("Order Id") * 100.0 / 
        (SELECT COUNT(*) 
         FROM supply_chain_data
         WHERE TO_CHAR("order date (DateOrders)"::timestamp, 'YYYY-MM') = '{year_month}')
       ) AS percentage
FROM supply_chain_data
WHERE TO_CHAR("order date (DateOrders)"::timestamp, 'YYYY-MM') = '{year_month}'
      AND "Shipping Mode" = '{shipping_mode}'
      AND "SUSPECTED_FRAUD" = {suspected_fraud}
GROUP BY "Customer Fname", "Customer Lname"
ORDER BY order_count DESC;
"""

# Execute the query and store the result in a DataFrame
try:
    result = pd.read_sql_query(query, con=engine)
    display(result)
except Exception as e:
    print(f"Error executing query: {e}")

,Customer Fname,Customer Lname,order_count,percentage
0,Azalia,Watson,1,0.047103
1,Blossom,Simon,1,0.047103
2,Germane,Short,1,0.047103
3,Jena,Ferrell,1,0.047103
4,Jenny,Kramer,1,0.047103
5,Medge,Mcfarland,1,0.047103
6,September,Massey,1,0.047103
7,Simone,Vance,1,0.047103


In [34]:
suspected_fraud = 'True'

# Query to calculate the total sales impacted by suspected fraud and categories with the highest suspected fraud
query = f"""
-- Total Sales Impacted by Suspected Fraud
SELECT SUM("Sales") AS total_sales_suspected_fraud
FROM supply_chain_data
WHERE "SUSPECTED_FRAUD" = {suspected_fraud};

-- Categories with the Highest Suspected Fraud
SELECT "Category Name", SUM("Sales") AS total_sales_suspected_fraud
FROM supply_chain_data
WHERE "SUSPECTED_FRAUD" = {suspected_fraud}
GROUP BY "Category Name"
ORDER BY total_sales_suspected_fraud DESC;
"""

# Execute the query and store the result in a DataFrame
try:
    result = pd.read_sql_query(query, con=engine)
    display(result)
except Exception as e:
    print(f"Error executing query: {e}")

,Category Name,total_sales_suspected_fraud
0,Fishing,157592.124334
1,Cleats,101563.072827
2,Camping & Hiking,91493.903355
3,Cardio Equipment,80202.059951
4,Women's Apparel,70450.000000
5,Men's Footwear,67074.842838
6,Water Sports,65796.711810
7,Indoor/Outdoor Games,65323.859366
8,Shop By Sport,28702.970440
9,Computers,16500.000000


In [35]:
query = f"""
SELECT "Year", 
       SUM(CASE WHEN "SUSPECTED_FRAUD" = {suspected_fraud} THEN "Sales" ELSE 0 END) AS fraud_sales,
       SUM("Sales") AS total_sales,
       (SUM(CASE WHEN "SUSPECTED_FRAUD" = {suspected_fraud} THEN "Sales" ELSE 0 END) * 100.0 / 
        NULLIF(SUM("Sales"), 0)) AS fraud_percentage
FROM supply_chain_data
GROUP BY "Year"
ORDER BY "Year";
"""

# Execute the query and store the result in a DataFrame
try:
    result = pd.read_sql_query(query, con=engine)
    display(result)
except Exception as e:
    print(f"Error executing query: {e}")

,Year,fraud_sales,total_sales,fraud_percentage
0,2015,267348.935047,1.234083e+07,2.166377
1,2016,266382.235309,1.230382e+07,2.165037
2,2017,285056.325332,1.180844e+07,2.414006
3,2018,7147.460093,3.316501e+05,2.155121
